# Very simple agent

This notebook show how to create a very simple conversation agent, with no tools.

### Let's start by importing the necessary libraries and set the api keys

In [1]:
from dotenv import load_dotenv

from typing import List
from agente.core.base import BaseAgent

## Load and set environment variables from .env file
load_dotenv()

True

### This first example show how to create an agent without streaming.

We note that even the agent is set to run without stream, we need to iterate over agent.run() to get the messages. 
This allows to stream in case a second agent with stream is called by the first parent agent.

In [2]:

class SimpleAgent(BaseAgent):
    agent_name: str = "SimpleAgent"
    system_prompt: str = "You are a helpful AI assistant."
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": False,
        "temperature": 1.0,
        "max_tokens": 500,
    }

# Create agent instance
agent = SimpleAgent()

# Add a message
agent.add_message(role = "user", content =  "Tell me a joke about programming.")

In [3]:
responses = await agent.run()

Executing agent: SimpleAgent


In [4]:
responses[0].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_71531/632561186.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[0].dict()


{'id': 'chatcmpl-CKKr02xxHIkx4KdPQ44WlbroAOvW6',
 'created': 1758962758,
 'model': 'gpt-4.1-2025-04-14',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_3502f4eb73',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None,
    'annotations': []},
   'provider_specific_fields': {}}],
 'usage': {'completion_tokens': 12,
  'prompt_tokens': 25,
  'total_tokens': 37,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0,
   'text_tokens': None},
  'prompt_tokens_details': {'audio_tokens': 0,
   'cached_tokens': 0,
   'text_tokens': None,
   'image_tokens': None}},
 'service_tier': 'default'}

### You can check the conversation history in the conv_history attribute:

In [5]:
all_messasges = agent.conv_history.messages

# Print the last response
for m in all_messasges:
    print(f"{m.role}: {m.content[0].text}")
    

system: You are a helpful AI assistant.
user: Tell me a joke about programming.
assistant: Why do programmers prefer dark mode?

Because light attracts bugs!


## Streaming

### Now let's create the same agent but with stream. For that we can use Gradio for better visualization

In [ ]:
from litellm import ModelResponseStream
from agente.models.schemas import StreamResponse
import gradio as gr


class VerySimpleAgent(BaseAgent):
    agent_name: str = "VerySimpleAgent"
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": True,
        "temperature": 1.0,
        "max_tokens": 500,
    }


def get_new_agent():
    """Create a fresh agent instance"""
    new_agent = VerySimpleAgent()
    return new_agent

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Initialize with a function call instead of direct instantiation
    main_agent = gr.State(value=None)


    def user(user_message, history, agent):
        if agent is None:
            agent = get_new_agent()
        agent.add_message("user", user_message)
        # Append to existing history instead of creating new list
        history = history or []
        history.append({"role": "user", "content": user_message})
        return "", history, agent

    async def bot(history, agent):
        if agent is None:
            agent = get_new_agent()
        if not history:
            yield [], agent
            return
        
        history.append({"role": "assistant", "content": ""})
        response_generator = await agent.run(stream=True)
        async for chunk in response_generator:
            #check if chunk is litellm or agente
            if isinstance(chunk, ModelResponseStream):
                if chunk.choices[0].delta.content:
                    history[-1]["content"] += chunk.choices[0].delta.content
                    yield history, agent
            elif isinstance(chunk, StreamResponse):
                if chunk.content:
                    history[-1]["content"] += chunk.content
                    yield history, agent
            else:
                raise ValueError(f"Unknown chunk type: {type(chunk)}")

    def reset_state():
        return None, get_new_agent()

    msg.submit(user, [msg, chatbot, main_agent], [msg, chatbot, main_agent]).then(
        bot, [chatbot, main_agent], [chatbot, main_agent]
    )
    clear.click(reset_state, None, [chatbot, main_agent])

demo.launch()

/Users/miguelwon/opt/miniconda3/envs/agente/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Executing agent: VerySimpleAgent
Executing agent: VerySimpleAgent


# Now let's check an agent using a tool

In [7]:
from agente.core.decorators import function_tool
import random

##### We create a very simple agent that can call a tool to get a random topic.

In [8]:

class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    @function_tool 
    def random_topic(self):
        """Tool to get a random topic.
        """
        topics = ["programming","science","animals","food","sports"]
        topic = random.choice(topics)

        return topic

##### We will use claude this time with reasoning enabled.

In [9]:

main_agent = MainAgent()
main_agent.completion_kwargs["model"] = "claude-sonnet-4-20250514"
main_agent.completion_kwargs["thinking"] = {"type": "enabled", "budget_tokens": 1024}
main_agent.add_message(role = "user", content = "Generate a random topic.")
responses = await main_agent.run()

Executing agent: main_agent
Executing tool: random_topic (agent: main_agent)
Executing agent: main_agent


##### The agent ran the first thinking step and then called the tool. The conversation history will all the steps.

In [10]:
for m in main_agent.conv_history.messages:
    print(m)
    print("--------------------------------")

role='user' agent_name='main_agent' content=[Content(type='text', text='Generate a random topic.')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 8, 46, 20, 914144)
--------------------------------
role='assistant' agent_name='main_agent' content=[ContentThinking(type='thinking', thinking='The user is asking for a random topic. I have a tool called "random_topic" that can generate a random topic. Let me check the parameters - it appears to have no required parameters (empty properties object), so I can call it directly.', signature='EpMDCkYICBgCKkBW543Q5L+sqgJ3oAGQQ0ECdTjmWxOh9Z0EFN51676HWooyCYarathRsSeWxNrCiQOc5iVfynaDqQpvQgqnmENbEgxYB3q/d2usi9xFWxsaDKUkwxYAzZlcbQ54sSIwmMNLZ6csoP/oZ5Qhs94CyMEsBSO39qYfY0GG5DpOWJuJtKc0+CYA9dAUd1WY1QcDKvoB8EDURGSalJ8XsZ4kAEfweGo/yybJoIYbqgC/did78EW2zmnBCmHmF7vXh25mA1DZNyqKFnYv3UmGkP+WhpSiIxdWwQSHw/9kaq6yKdPr1ucPJuBS5R+EzsJ5siQPfQC6MDd18aMm6ktdJaQSRCrCLtVec0nBMyhHzUi36GdVz+1FvBirP

In [11]:
main_agent.conv_history.messages[0].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_71531/800015913.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[0].dict()


{'role': 'user',
 'agent_name': 'main_agent',
 'content': [{'type': 'text', 'text': 'Generate a random topic.'}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': None,
 'timestamp': datetime.datetime(2025, 9, 27, 8, 46, 20, 914144)}

In [12]:
main_agent.conv_history.messages[1].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_71531/3833547919.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[1].dict()


{'role': 'assistant',
 'agent_name': 'main_agent',
 'content': [{'type': 'thinking',
   'thinking': 'The user is asking for a random topic. I have a tool called "random_topic" that can generate a random topic. Let me check the parameters - it appears to have no required parameters (empty properties object), so I can call it directly.',
   'signature': 'EpMDCkYICBgCKkBW543Q5L+sqgJ3oAGQQ0ECdTjmWxOh9Z0EFN51676HWooyCYarathRsSeWxNrCiQOc5iVfynaDqQpvQgqnmENbEgxYB3q/d2usi9xFWxsaDKUkwxYAzZlcbQ54sSIwmMNLZ6csoP/oZ5Qhs94CyMEsBSO39qYfY0GG5DpOWJuJtKc0+CYA9dAUd1WY1QcDKvoB8EDURGSalJ8XsZ4kAEfweGo/yybJoIYbqgC/did78EW2zmnBCmHmF7vXh25mA1DZNyqKFnYv3UmGkP+WhpSiIxdWwQSHw/9kaq6yKdPr1ucPJuBS5R+EzsJ5siQPfQC6MDd18aMm6ktdJaQSRCrCLtVec0nBMyhHzUi36GdVz+1FvBirP2vSFJl9SW5EiwLWtvHKfXcyCONLHF5KIlghRcr8JvDlKnbc5ihy2DtXTjd5lIWUedZbKPnGMEiRaE8M/MTQBh/SFQ11S+IiysJKZ3BYtRwZobeWdYQgf2RxIAYVGSTaHe+N+cgGP9NWBIi6CM8CvO+zP2oKMBgB'}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 

##### If needed, we can call again to continue the conversation.

In [13]:
main_agent.add_message(role = "user", content = "Another one.")
responses = await main_agent.run(max_retries=10)

Executing agent: main_agent
Executing tool: random_topic (agent: main_agent)
Executing agent: main_agent


In [14]:
main_agent.conv_history.messages[-1].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_71531/1887248032.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[-1].dict()


{'role': 'assistant',
 'agent_name': 'main_agent',
 'content': [{'type': 'text',
   'text': "Your random topic is: **Programming**\n\nThis topic encompasses a vast field including programming languages, software development, algorithms, web development, mobile apps, artificial intelligence, debugging, coding best practices, and much more. Whether you're interested in learning to code, discussing different programming paradigms, or exploring the latest tech trends, there's plenty to dive into with this topic!"}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': {'completion_tokens': 78, 'prompt_tokens': 593, 'total_tokens': 671},
 'timestamp': datetime.datetime(2025, 9, 27, 8, 46, 31, 492988)}